<a href="https://colab.research.google.com/github/LUPRWEST/WEB/blob/main/food_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
# 내 드라이브 > data > kaggle_covid19 폴더 만들기
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/My Drive/data/kaggle_food/"

In [ ]:
# 구글 드라이브 접근 권한 받기 
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
%cd /content/gdrive/My Drive/data/kaggle_food

[Errno 2] No such file or directory: '/content/gdrive/My Drive/data/kaggle_food'
/content


In [ ]:
!ls

image


In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPool2D
from keras.preprocessing.image import ImageDataGenerator

from tensorflow import keras
import matplotlib.pyplot as plt
import numpy
import os 
import tensorflow as tf
import cv2

In [ ]:
from keras.utils import np_utils

In [18]:
# ImageDataGenerator 
train_datagen = ImageDataGenerator(rescale= 1./255)
training_set = train_datagen.flow_from_directory('image/train',
                                                 target_size= (288, 288),
                                                 batch_size= 32,
                                                 class_mode= 'categorical')

Found 250 images belonging to 5 classes.


In [21]:
test_datagen = ImageDataGenerator(rescale= 1./255)
test_set = test_datagen.flow_from_directory('image/test',
                                            target_size= (288, 288),
                                            batch_size= 32,
                                            class_mode= 'categorical')

Found 1009 images belonging to 5 classes.


In [38]:
#그거 뭐냐 레이블 나누기
test_set.classes = ['bibimbap', 'caprese_salad', 'cup_cakes', 'hamburger', 'pizza']

from sklearn.preprocessing import LabelEncoder
item = ['bibimbap', 'caprese_salad', 'cup_cakes', 'hamburger', 'pizza']

encoder = LabelEncoder()
encoder.fit(test_set.classes)
result = encoder.transform(test_set.classes)

target = tf.keras.utils.to_categorical(result)
print(target)

[[1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1.]]


In [39]:
#model design

model = Sequential()
model.add(Conv2D(32, (3, 3), activation = 'relu', input_shape = (288,288, 3)))
model.add(MaxPool2D(pool_size=(2, 2), strides=2))

model.add(Conv2D( 32,(3, 3), activation = 'relu'))
model.add(MaxPool2D(pool_size=(2, 2), strides=2))

# fully-connected layer 

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [24]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 286, 286, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 143, 143, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 141, 141, 32)      9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 70, 70, 32)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 156800)            0         
                                                                 
 dense (Dense)               (None, 128)               2

In [40]:
# (실습) model compile : 손실함수('binary_crossentropy')
# 최적화 함수 : SGD(학습률 0.01)
from tensorflow.keras.optimizers import SGD
model.compile(loss= 'categorical_crossentropy',
              optimizer= 'adam', 
              metrics=['accuracy'])

In [41]:
# (실습) model training : 이미지데이터생성으로 만든 training_set 데이터를 25번 학습함. 
history = model.fit(training_set, epochs=5)
#폴더명이 레이블이기 때문에 따로 레이블 써주지 x여도 ok

#early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
#history = model.fit(training_set, validation_data = test_set, epochs = 25, callbacks=[early_stop])

Epoch 1/5


InvalidArgumentError: ignored

In [ ]:
# 정확도(accuracy), 손실(loss) 그래프로 표현 
plt.figure(1)
plt.xlabel('epoch')
plt.plot(history.history['accuracy'], label='training_accuracy')
plt.legend()
plt.show()

plt.figure(2)
plt.xlabel('epoch')
plt.plot(history.history['loss'], label='training_loss')
plt.legend()
plt.show()

In [ ]:
# (실습) 5. model evaluate(test_data, test_target) 
test_loss, test_acc = model.evaluate(test_data, test_target) 
print(test_loss)
print(test_acc)

In [ ]:
# (실습) 6. 훈련된 모델 이용해서 이미지 예측하기(prediction)
predictions = model.predict(test_data)

In [ ]:
# 10개의 숫자에 상응하는 모델의 신뢰도(confidence)
# 첫번째 예측
predictions[3]

In [ ]:
# 신뢰도가 가장 높은 레이블 
import numpy as np
np.argmax(predictions[3])